<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_ANN_BCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving BCDD.csv to BCDD.csv


In [2]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['BCDD.csv']),header=None)
df2.head()

,0,1,2,3,4,5,6,7
0,1.0,35.0,1.0,11753.0,2.0,4.0,38.0,1.0
1,1.0,24.0,1.0,5835.0,7.0,2.0,68.0,1.0
2,1.0,10.0,1.0,9020.0,11.0,14.0,63.0,1.0
3,1.0,39.0,1.0,9425.0,7.0,0.0,73.0,1.0
4,1.0,17.0,1.0,15672.0,42.0,0.0,43.0,1.0


In [0]:
features =df2.drop(df2.columns[7],axis=1)
labels=df2[df2.columns[7]]

In [4]:
import numpy as np
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[0.e+00 1.e+00]
 [5.e+04 5.e+04]]


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[-0.83985099  0.71924274 -0.18851982  0.79273848 -1.15437249 -0.06768473
  1.35873136]
(80000, 7)
(20000, 7)
(80000,)
(20000,)


In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()
model.add(Dense(200,input_shape=(7,),activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m,precision_m, recall_m])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 80000 samples, validate on 20000 samples
Epoch 1/10





80000/80000 [==============================] - 4s 54us/step - loss: 0.2668 - acc: 0.8877 - f1_m: 0.8861 - precision_m: 0.8747 - recall_m: 0.9043 - val_loss: 0.2640 - val_acc: 0.8857 - val_f1_m: 0.8825 - val_precision_m: 0.8844 - val_recall_m: 0.8874
Epoch 2/10
80000/80000 [==============================] - 4s 46us/step - loss: 0.2597 - acc: 0.8888 - f1_m: 0.8875 - precision_m: 0.8747 - recall_m: 0.9073 - val_loss: 0.2621 - val_acc: 0.8871 - val_f1_m: 0.8841 - val_precision_m: 0.8856 - val_recall_m: 0.8894
Epoch 3/10
80000/80000 [==============================] - 4s 48us/step - loss: 0.2590 - acc: 0.8885 - f1_m: 0.8873 - precision_m: 0.8746 - recall_m: 0.9070 - val_loss: 0.2609 - val_acc: 0.8880 - val_f1_m: 0.8877 - val_precision_m: 0.8689 - val_recall_m: 0.9144
Epoch 4/10
80000/80000 [==============================] - 4s 47us/step - loss: 0.2588 - acc: 0.8891 - f1_m: 0.8876 - precision_m: 0.8751 - recall_m: 0.9070 - v

In [9]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 26.02%
acc: 88.87%
f1: 88.69%
precision: 87.84%
recall: 90.23%


In [0]:
y_pred=model.predict_classes(X_test)

In [11]:
from sklearn import metrics
print("Accuracy: "+"{:.2%}".format(metrics.accuracy_score(y_test, y_pred)));
print("Precision: "+"{:.2%}".format(metrics.precision_score(y_test, y_pred)));
print("Recall: "+"{:.2%}".format(metrics.recall_score(y_test, y_pred)));
print("F1: "+"{:.2%}".format(metrics.f1_score(y_test,y_pred)));
print(metrics.confusion_matrix(y_test,y_pred));

Accuracy: 88.87%
Precision: 87.87%
Recall: 90.27%
F1: 89.05%
[[8714 1251]
 [ 976 9059]]
